## Preface
In the project, we extract data from amazon, and prepare for further analysis. Also, we try to visualize data in order to describe consumptional behaviors.


## Collect data
We use BeautifulSoup to parse the raw HTML. The variables for each items are: image data in bytes, image link, item link, name of goods, numbers of rating, price and rating(0-5).

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# one page item
def getOnePage(url):
    onepage = []
    headers = {
        'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'
        }
    # call response
    response = requests.get(url, headers=headers)
    # build Soup object 
    root = BeautifulSoup(response.text, 'html5lib')
    # get a list contain each item's HTML
    items = root.find_all("div", class_="s-item-container")  
    
    for i in items:
        try:
            # deal with each item HTML
            d = {}
            d['img_data'] = i.find("img")['src']
            d['img_link'] = i.find("img")['src']
            d['link'] = i.find("a", class_="s-access-detail-page")['href']
            d['name'] = i.find("a", class_="s-access-detail-page").text
            d['num_ratings'] = \
            float((i.find("a", {"href":lambda x : x.endswith("#customerReviews")}).text).replace(',',''))
            d['price'] = \
            float(i.find("span", class_="sx-price-whole").text + '.'+\
            i.find("sup",class_="sx-price-fractional").text
            )
            d['stars'] = i.find("i",class_="a-icon-star").text
            # print(d)
            onepage.append(d)
        except (TypeError, AttributeError):
            pass
    return onepage
    
# url = 'https://www.amazon.com/s/ref=sd_allcat_nav_desktop_sa_intl_baby_toddler_toys?_encoding=UTF8&bbn=16225005011&rh=i%3Aspecialty-aps%2Cn%3A%2116225005011%2Cn%3A196601011'
# getOnePage(url)

In [3]:
def getUrl(page):
    return 'https://www.amazon.com/s/ref=sr_pg_'+str(page)+\
    '?rh=i%3Ababy-products-intl-ship%2Cn%3A%2116225005011%2Cn%3A196601011&page='+str(page)+\
    '&bbn=16225005011&ie=UTF8&qid=1549443515'

In [ ]:
# all scapling
def scaplingAll():
    ans = []
    page = 1
    while True and page<500:
        url = getUrl(page)
        #print(url)
        try:
            onePageItems = getOnePage(url)
            ans = ans + onePageItems # add items
            page = page + 1
        except (SysCallError, SSLError):
            break
    return ans


items = scaplingAll()



In [ ]:
import pandas as pd

# finally get the list
df = pd.DataFrame(items)

df.to_csv('amazon_toys.csv', encoding='utf-8', index=False)



In [4]:
df = pd.read_csv('amazon_toys.csv')
df.head()

,img_data,img_link,link,name,num_ratings,price,stars
0,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/ORWINE-Educational-Fric...,ORWINE Inertia Toy Early Educational Toddler B...,99.0,16.99,4.8 out of 5 stars
1,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/Melissa-Doug-Flip-Fish-...,"Melissa & Doug Flip Fish Baby Toy, Development...",243.0,14.99,4.8 out of 5 stars
2,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/TYRY-HU-Pacifier-Silico...,TYRY.HU Pacifier Clips Silicone Teething Beads...,209.0,13.99,4.1 out of 5 stars
3,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/Patrol-Ultimate-Rescue-...,PAW Patrol Ultimate Rescue Chase's Mini Helico...,45.0,7.29,4.8 out of 5 stars
4,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com/Skip-Hop-Explore-Follow...,Skip Hop Explore & More Follow-Me Bee Crawl To...,131.0,23.49,4.5 out of 5 stars


Now, we get the final data frame object using for further analysis.